<a href="https://colab.research.google.com/github/DNThuan/AspectBasedSentimentAnalysis/blob/main/ABSA_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import library

In [1]:
!git clone -q https://github.com/DNThuan/AspectBasedSentimentAnalysis.git
!pwd
!ls

/content
AspectBasedSentimentAnalysis  sample_data


In [2]:
!pip install -q emoji
!pip install pyvi

     |████████████████████████████████| 170 kB 5.5 MB/s 
     |████████████████████████████████| 8.5 MB 4.4 MB/s 
     |████████████████████████████████| 743 kB 44.0 MB/s 


In [3]:
import numpy as np
import json
import pandas as pd
import re
import emoji

from sklearn.preprocessing import  FunctionTransformer
from pyvi import ViTokenizer, ViPosTagger
from AspectBasedSentimentAnalysis.utils import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

#2. Preprocessing data

In [4]:
def show_label(list_label, pred_as, pred_pos, pred_neg, pred_neu):
  labels = []
  for index, value in enumerate(pred_as):
    if value == 1:
      label = "{"
      if pred_pos[index] ==1:
        label += str(list_label[index])+", "+"positive"
      elif pred_neg[index] ==1:
        label += str(list_label[index])+", "+"negative"
      elif pred_neu[index] ==1:
        label += str(list_label[index])+", "+"neutral"
      label+="}"
      labels.append(label)
  return labels

In [5]:

def read_label(path):
  with open(path) as f:
    data = json.load(f)
  f.close()
  return data

aspect_path = "/content/AspectBasedSentimentAnalysis/Label/aspect.json"
SA_path = "/content/AspectBasedSentimentAnalysis/Label/SA.json"


aspect_labels = read_label(aspect_path)
AS_labels = read_label(SA_path)

transform_label_aspect = MultiLabelBinarizer().fit([aspect_labels])
list_label_aspect = transform_label_aspect.classes_

transform_label_SA = MultiLabelBinarizer().fit([AS_labels])
list_label_SA = transform_label_SA.classes_

## 2.1 Get data

In [6]:
def read_data(path):
  with open(path) as f:
    data = f.read().split("\n\n")
  f.close()
  for i in range(len(data)):
    temp=data[i].split("\n")
    data[i]=temp[1:3]
  return data

In [7]:
path_train = "/content/AspectBasedSentimentAnalysis/VLSP2018_SA_Hotel/1-VLSP2018-SA-Hotel-train (7-3-2018).txt"
path_dev =  "/content/AspectBasedSentimentAnalysis/VLSP2018_SA_Hotel/2-VLSP2018-SA-Hotel-dev (7-3-2018).txt"
path_test = "/content/AspectBasedSentimentAnalysis/VLSP2018_SA_Hotel/3-VLSP2018-SA-Hotel-test (8-3-2018).txt"

train = np.array(read_data(path_train))
dev = np.array(read_data(path_dev))
test = np.array(read_data(path_test))

print("Train: ",train.shape)
print("Dev: ",dev.shape)
print("Test: ",test.shape)


Train:  (3000, 2)
Dev:  (2000, 2)
Test:  (600, 2)


In [8]:
X_train, y_train = train[:,0], train[:,1]
X_dev,   y_dev   = dev[:,0],   dev[:,1]
X_test,  y_test  = test[:,0],  test[:,1]

In [40]:
print(X_train)

['Rộng rãi KS mới nhưng rất vắng. Các dịch vụ chất lượng chưa cao và thiếu.'
 'Địa điểm thuận tiện, trong vòng bán kính 1,5km nhiều quán ăn ngon'
 'Phục vụ, view đẹp, vị trí' ...
 'Xin chào Chudu24,Xin cám ơn dịch vụ booking của Chudu24 đã cho vợ chồng tôi có một kỳ nghỉ yên tâm, tin tưởng và thật thoải mái. ôi xin có một số đánh giá sau đây về Sapa View hotel:1. Phòng Deluxe garden view: Phòng sạch sẽ, tiện nghi, yên tĩnh, gần trung tâm và đặc biệt view rất đẹp nhìn xuống thung lũng với ruộng bậc thang mênh mông cùng với balcony rất ấn tượng cùng giỏ hoa mầu sắc. Do ở HN chỉ có điều hòa 2 chiều nên khi ở đó mình được trải nhiệm với lò sưởi bằng củi rất đặc trưng Sapa. Buổi tối ngồi bàn đôi cạnh lò sưởi cùng ánh nến ấm áp và li rượu vang thật là tuyệt vời ông mặt zời. 2. Phục vụ: Nhân viên phục vụ là mấy em người Mông nên rất là thật thà, chu đáo. Theo quy định 2h mới được nhận phòng nhưng do đến SaPa lúc 8AM bà chủ Sapa View đã chào đón niềm nở và cho nhận phòng luôn.3. Ăn uống: Thực 

In [41]:
print(y_train)

['{HOTEL#DESIGN&FEATURES, positive}, {HOTEL#GENERAL, negative}'
 '{LOCATION#GENERAL, positive}'
 '{SERVICE#GENERAL, positive}, {HOTEL#GENERAL, positive}, {LOCATION#GENERAL, positive}'
 ...
 '{ROOMS#CLEANLINESS, positive}, {ROOMS#COMFORT, positive}, {LOCATION#GENERAL, positive}, {ROOMS#GENERAL, positive}, {SERVICE#GENERAL, positive}, {FOOD&DRINKS#STYLE&OPTIONS, negative}, {ROOMS#DESIGN&FEATURES, negative}, {HOTEL#PRICES, negative}, {FOOD&DRINKS#PRICES, negative}, {ROOM_AMENITIES#DESIGN&FEATURES, neutral}'
 '{ROOMS#CLEANLINESS, positive}, {ROOM_AMENITIES#DESIGN&FEATURES, positive}, {ROOMS#GENERAL, positive}, {LOCATION#GENERAL, negative}, {FOOD&DRINKS#STYLE&OPTIONS, negative}, {FOOD&DRINKS#QUALITY, negative}, {HOTEL#PRICES, negative}, {HOTEL#GENERAL, negative}'
 '{HOTEL#DESIGN&FEATURES, positive}, {HOTEL#COMFORT, positive}, {SERVICE#GENERAL, positive}, {HOTEL#GENERAL, positive}']


## 2.2 Preprocess review data

In [9]:
X_train_aspect_preprocess = preprocessing.Preprocessing_Review(X_train).process()

X_dev_aspect_preprocess = preprocessing.Preprocessing_Review(X_dev).process()

X_test_aspect_preprocess = preprocessing.Preprocessing_Review(X_test).process()

## 2.3 Covert to vector

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

TF_IDF = TfidfVectorizer(sublinear_tf=True, min_df=5, ngram_range=(1,3), stop_words='english').fit(X_train_aspect_preprocess)

X_train_aspect_tf = TF_IDF.transform(X_train_aspect_preprocess).toarray()
X_dev_aspect_tf = TF_IDF.transform(X_dev_aspect_preprocess).toarray()
X_test_aspect_tf = TF_IDF.transform(X_test_aspect_preprocess).toarray()

## 2.4 Preprocessing label for aspect

In [11]:
y_train_aspect_preprocess = preprocessing.Preprocessing_Label(y_train,list_label_aspect, transform_label_aspect, aspect = True).make_label_dataframe()

y_dev_aspect_preprocess = preprocessing.Preprocessing_Label(y_dev,list_label_aspect,transform_label_aspect,aspect = True).make_label_dataframe()

y_test_aspect_preprocess = preprocessing.Preprocessing_Label(y_test,list_label_aspect, transform_label_aspect, aspect = True).make_label_dataframe()

# 3. Modeling

In [12]:
from sklearn import metrics

def score(y_true, y_pred):
  print("Precison: ",metrics.precision_score(y_true, y_pred, average='micro'))
  print("Recall: ",metrics.recall_score(y_true, y_pred, average='micro'))
  print("F1: ",metrics.f1_score(y_true, y_pred, average='micro'))


In [13]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

clf_aspect = OneVsRestClassifier(LinearSVC(), n_jobs=1)

clf_positive = OneVsRestClassifier(LinearSVC(), n_jobs=1)
clf_negative= OneVsRestClassifier(LinearSVC(), n_jobs=1)
clf_neutral = OneVsRestClassifier(LinearSVC(), n_jobs=1)   

## 3.1 Aspect detection

In [14]:
clf_aspect.fit(X_train_aspect_tf, y_train_aspect_preprocess)


OneVsRestClassifier(estimator=LinearSVC(), n_jobs=1)

### 3.1.1 Predict aspect in train

In [15]:
train_predict_aspect =  clf_aspect.predict(X_train_aspect_tf)

print(classification_report(y_train_aspect_preprocess,train_predict_aspect, target_names=list_label_aspect))

                                precision    recall  f1-score   support

        FACILITIES#CLEANLINESS       1.00      1.00      1.00       172
            FACILITIES#COMFORT       1.00      0.98      0.99       132
    FACILITIES#DESIGN&FEATURES       0.99      0.99      0.99       525
            FACILITIES#GENERAL       1.00      0.95      0.97       219
      FACILITIES#MISCELLANEOUS       1.00      0.97      0.98        33
             FACILITIES#PRICES       1.00      0.96      0.98        53
            FACILITIES#QUALITY       1.00      0.97      0.98       208
     FOOD&DRINKS#MISCELLANEOUS       1.00      1.00      1.00        13
            FOOD&DRINKS#PRICES       1.00      0.99      1.00       118
           FOOD&DRINKS#QUALITY       1.00      1.00      1.00       672
     FOOD&DRINKS#STYLE&OPTIONS       1.00      1.00      1.00       569
             HOTEL#CLEANLINESS       1.00      0.99      0.99       348
                 HOTEL#COMFORT       0.99      0.99      0.99  

### 3.1.2 Predict aspect in dev

In [16]:
dev_predict_aspect =  clf_aspect.predict(X_dev_aspect_tf)

print(classification_report(y_dev_aspect_preprocess,dev_predict_aspect, target_names=list_label_aspect))

                                precision    recall  f1-score   support

        FACILITIES#CLEANLINESS       0.70      0.30      0.42        23
            FACILITIES#COMFORT       0.50      0.01      0.03        69
    FACILITIES#DESIGN&FEATURES       0.74      0.35      0.48       220
            FACILITIES#GENERAL       0.71      0.16      0.26        62
      FACILITIES#MISCELLANEOUS       0.00      0.00      0.00         7
             FACILITIES#PRICES       1.00      0.03      0.05        36
            FACILITIES#QUALITY       0.91      0.29      0.44       101
     FOOD&DRINKS#MISCELLANEOUS       0.00      0.00      0.00        11
            FOOD&DRINKS#PRICES       0.50      0.03      0.06        29
           FOOD&DRINKS#QUALITY       0.88      0.79      0.83       423
     FOOD&DRINKS#STYLE&OPTIONS       0.79      0.75      0.77       289
             HOTEL#CLEANLINESS       0.70      0.50      0.58       222
                 HOTEL#COMFORT       0.58      0.45      0.51  

### 3.1.3 Predict aspect in test

In [17]:
test_predict_aspect =  clf_aspect.predict(X_test_aspect_tf)

print(classification_report(y_test_aspect_preprocess,test_predict_aspect, target_names=list_label_aspect))

                                precision    recall  f1-score   support

        FACILITIES#CLEANLINESS       0.00      0.00      0.00         5
            FACILITIES#COMFORT       0.00      0.00      0.00        26
    FACILITIES#DESIGN&FEATURES       0.78      0.28      0.41        65
            FACILITIES#GENERAL       0.60      0.14      0.23        21
      FACILITIES#MISCELLANEOUS       0.00      0.00      0.00         8
             FACILITIES#PRICES       0.00      0.00      0.00        13
            FACILITIES#QUALITY       0.92      0.24      0.38        51
     FOOD&DRINKS#MISCELLANEOUS       0.00      0.00      0.00         3
            FOOD&DRINKS#PRICES       0.00      0.00      0.00         9
           FOOD&DRINKS#QUALITY       0.82      0.78      0.80       129
     FOOD&DRINKS#STYLE&OPTIONS       0.87      0.74      0.80       124
             HOTEL#CLEANLINESS       0.53      0.48      0.50        67
                 HOTEL#COMFORT       0.56      0.47      0.51  

## 3.2 Predict full

### 3.2.1 Merge predict aspect with X_train

In [18]:
X_train_polariry = np.hstack((X_train_aspect_tf,train_predict_aspect))

X_dev_polatity = np.hstack((X_dev_aspect_tf,dev_predict_aspect))

X_test_polatity = np.hstack((X_test_aspect_tf,test_predict_aspect))

### 3.2.1 Preproceessing label for polarity

In [19]:
y_train_positive = preprocessing.Preprocessing_Label(y_train, list_label_aspect,transform_label_aspect, positive = True).make_label_dataframe()

y_train_negative = preprocessing.Preprocessing_Label(y_train, list_label_aspect, transform_label_aspect,negative = True).make_label_dataframe()

y_train_neutral = preprocessing.Preprocessing_Label(y_train,list_label_aspect,transform_label_aspect,neutral = True).make_label_dataframe()


In [20]:
y_dev_positive = preprocessing.Preprocessing_Label(y_dev,list_label_aspect, transform_label_aspect,positive = True).make_label_dataframe()

y_dev_negative = preprocessing.Preprocessing_Label(y_dev,list_label_aspect,transform_label_aspect, negative = True).make_label_dataframe()

y_dev_neutral = preprocessing.Preprocessing_Label(y_dev, list_label_aspect,transform_label_aspect,neutral = True).make_label_dataframe()

In [21]:
y_test_positive = preprocessing.Preprocessing_Label(y_test,list_label_aspect, transform_label_aspect, positive = True).make_label_dataframe()

y_test_negative = preprocessing.Preprocessing_Label(y_test,list_label_aspect,transform_label_aspect, negative = True).make_label_dataframe()

y_test_neutral = preprocessing.Preprocessing_Label(y_test,list_label_aspect,transform_label_aspect, neutral = True).make_label_dataframe()

### 3.2.3 Fit

In [22]:
clf_positive.fit(X_train_polariry, y_train_positive)
clf_negative.fit(X_train_polariry, y_train_negative)
clf_neutral.fit(X_train_polariry, y_train_neutral)

OneVsRestClassifier(estimator=LinearSVC(), n_jobs=1)

### 3.2.4 Predict details polarity label of dev dataset

In [23]:
predict_positive = clf_positive.predict(X_test_polatity)
predict_negative = clf_negative .predict(X_test_polatity)
predict_neutral = clf_neutral.predict(X_test_polatity)

In [24]:
score(y_test_positive,predict_positive)

Precison:  0.6477987421383647
Recall:  0.5703211517165006
F1:  0.6065959952885748


In [25]:
score(y_test_negative,predict_negative)

Precison:  0.5656934306569343
Recall:  0.24031007751937986
F1:  0.337323177366703


In [26]:
score(y_test_neutral,predict_neutral)

Precison:  0.0
Recall:  0.0
F1:  0.0


# 4. Predict full

In [27]:
from AspectBasedSentimentAnalysis.utils.preprocessing import pre_label

In [28]:
def predict_full(X):
  predict_aspect = clf_aspect.predict(X)
  predict_positive = clf_positive.predict(np.hstack((X,predict_aspect)))
  predict_negative = clf_negative.predict(np.hstack((X,predict_aspect)))
  predict_neutral = clf_neutral.predict(np.hstack((X,predict_aspect)))

  predict_full =[]
  for i in range(len(X)):
    pred = show_label(list_label_aspect,predict_aspect[i],predict_positive[i],predict_negative[i],predict_neutral[i])
    predict_full.append(pred)

  return predict_full

### 4.1 Dev dataset

In [31]:
  y_dev_pred_full = predict_full(X_dev_aspect_tf)
  y_dev_pred_full = transform_label_SA.transform(y_dev_pred_full)

  _,_, y_dev_true = pre_label.separate_label(y_dev)
  y_dev_true_tf = transform_label_SA.transform(y_dev_true)


In [34]:
print(classification_report(y_dev_true_tf,y_dev_pred_full, target_names=list_label_SA))

                                            precision    recall  f1-score   support

        {FACILITIES#CLEANLINESS, negative}       0.00      0.00      0.00         8
         {FACILITIES#CLEANLINESS, neutral}       0.00      0.00      0.00         0
        {FACILITIES#CLEANLINESS, positive}       0.50      0.33      0.40        15
            {FACILITIES#COMFORT, negative}       0.00      0.00      0.00        37
             {FACILITIES#COMFORT, neutral}       0.00      0.00      0.00         0
            {FACILITIES#COMFORT, positive}       0.00      0.00      0.00        32
    {FACILITIES#DESIGN&FEATURES, negative}       0.57      0.03      0.06       119
     {FACILITIES#DESIGN&FEATURES, neutral}       0.00      0.00      0.00         8
    {FACILITIES#DESIGN&FEATURES, positive}       0.31      0.31      0.31        93
            {FACILITIES#GENERAL, negative}       0.00      0.00      0.00         3
             {FACILITIES#GENERAL, neutral}       0.00      0.00      0.00  

## 4.2 Test dataset

In [35]:
  y_test_pred_full = predict_full(X_test_aspect_tf)
  y_test_pred_full = transform_label_SA.transform(y_test_pred_full)

  _,_, y_test_true = pre_label.separate_label(y_test)
  y_test_true_tf = transform_label_SA.transform(y_test_true)


In [36]:
print(classification_report(y_test_true_tf,y_test_pred_full,target_names=list_label_SA))

                                            precision    recall  f1-score   support

        {FACILITIES#CLEANLINESS, negative}       0.00      0.00      0.00         3
         {FACILITIES#CLEANLINESS, neutral}       0.00      0.00      0.00         0
        {FACILITIES#CLEANLINESS, positive}       0.00      0.00      0.00         2
            {FACILITIES#COMFORT, negative}       0.00      0.00      0.00         6
             {FACILITIES#COMFORT, neutral}       0.00      0.00      0.00         0
            {FACILITIES#COMFORT, positive}       0.00      0.00      0.00        20
    {FACILITIES#DESIGN&FEATURES, negative}       0.00      0.00      0.00        22
     {FACILITIES#DESIGN&FEATURES, neutral}       0.00      0.00      0.00         5
    {FACILITIES#DESIGN&FEATURES, positive}       0.39      0.24      0.30        38
            {FACILITIES#GENERAL, negative}       0.00      0.00      0.00         1
             {FACILITIES#GENERAL, neutral}       0.00      0.00      0.00  

# 5. Predict for text file (.txt)

## 5.1 Read file text

In [ ]:
path_file = "/content/VLSP_Hotel_test.txt"
texts = read_data(path_file)
print(len(texts))


In [ ]:
texts

In [ ]:
texts = [x[0] for x in texts]
texts_preprocess = preprocessing.Preprocessing_Review(texts,
                                              lower_text=True,
                                              delete_emoji=True,
                                              replace_symbol=True,
                                              delete_special_character = True,
                                              replace_negative_words=True,
                                              normalize_elongate_words=True
                                              ).process()
texts_tf = TF_IDF.transform(texts_preprocess).toarray()

## 5.2 Predict

In [ ]:
predicts = predict_full(texts_tf)
file = open("file_predict.txt","w")
with file as f:
  for index,predict in enumerate(predicts):
    f.write("#"+str(index+1)+"\n")
    f.write(texts[index]+"\n")
    for i in predict:
      f.write(i+", ")
    f.write("\n")
f.close()